In [4]:
import pandas as pd

In [5]:
from sklearn.feature_extraction import DictVectorizer

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [4]:
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet')

In [5]:
len(df.columns)

19

In [6]:
df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    # Converting to Minutes
df["duration"] = df.duration.apply(lambda td: td.total_seconds()/60)
df.duration.describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [7]:
((df.duration >= 1) & (df.duration <= 60)).mean()

0.9812202822125979

In [7]:
# Function that reads parquet and converts the date time to duration in minutes
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    # Converting to Minutes
    df["duration"] = df.duration.apply(lambda td: td.total_seconds()/60)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    return df

In [8]:
# Read training and validation files
df_train = read_dataframe("./data/yellow_tripdata_2023-01.parquet")
df_val = read_dataframe("./data/yellow_tripdata_2023-02.parquet")

In [10]:
len(df_train), len(df_val)

(3009173, 2855951)

In [9]:
#Define the features to be used
categorical = ["PULocationID","DOLocationID"]
numerical = ["trip_distance"]
df_train[categorical] = df_train[categorical].astype(str)
df_val[categorical] = df_val[categorical].astype(str)
#One-hot encoding
dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient="records")

X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient="records")

X_val = dv.transform(val_dicts)



In [10]:
X_train

<3009173x516 sparse matrix of type '<class 'numpy.float64'>'
	with 9027519 stored elements in Compressed Sparse Row format>

In [11]:
# Values to be predicted
target = "duration"
y_train = df_train[target].values
y_val= df_val[target].values

In [ ]:
#LR Model
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

In [13]:
#LR Model
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.811472580390592